In [1]:
from math import radians, sin, sqrt

import cadquery as cq
import ipywidgets as widgets
import jupyter_cadquery as jcq
from jupyter_cadquery.replay import replay, show

Overwriting auto display for cadquery Workplane and Shape


In [2]:
jcq.set_defaults(
    control="orbit",
    axes=True,
    axes0=True,
    grid=(True, False, False),
    glass=True,
    theme="browser",
)

# open_viewer("Cadquery")
replay_box = widgets.Checkbox(False, description="Enable Replay")
out = widgets.Output()
display(replay_box)
display(out)


@out.capture()
def changed(e):
    if e["new"]:
        jcq.replay.enable_replay(warning=False)
    else:
        jcq.replay.disable_replay()


replay_box.observe(changed, names="value")

Checkbox(value=False, description='Enable Replay')

Output()

In [3]:
# 45, 22
extrusions_w = 20
nozzle_z_distance = 40
width = 40
screw_d = 5
screw_head_d = 9
screw2_d = 3 - 0.5
screw2_head_d = 6
screw2_head_h = 3
steel_wire_d = 0.7 + 0.1
thickness = 4
min_thickness = 2

In [58]:
# 6 7 35

logo = (
    cq.importers.importDXF("voron_logo.dxf").wires().toPending().extrude(100)
)
replay(logo)
cq.exporters.export(logo, "/tmp/logo.stl")

Use the multi select box below to select one or more steps you want to examine


In [59]:
guide_height = min_thickness * 2 + screw2_head_d
height = extrusions_w + nozzle_z_distance + guide_height / 2
screw1_z = height / 2 - extrusions_w / 2
screw1_x = width / 2 - screw_head_d / 2 - thickness
screw1_pos = [(screw1_x, screw1_z), (-screw1_x, screw1_z)]
guide_pos = -height / 2 + guide_height / 2
screw2_x_pos = width / 2 - guide_height / 2
steel_hole_x_pos = screw2_x_pos + screw2_head_d / 2

r = (
    cq.Workplane("XY")
    .box(width, thickness, height)
    .faces(">Y")
    .workplane()
    .pushPoints([(0, guide_pos)])
    .rect(width, guide_height)
    .extrude(10)
    .chamfer(1)
    .faces("<Y")
    .pushPoints(screw1_pos)
    .hole(screw_d)
    .faces(">Y")
    .workplane(centerOption="CenterOfMass")
    .pushPoints([(-screw2_x_pos, 0), (screw2_x_pos, 0)])
    .cboreHole(screw2_d, screw2_head_d, screw2_head_h)
    .pushPoints([(-steel_hole_x_pos, 0), (steel_hole_x_pos, 0)])
    .hole(steel_wire_d)
)

replay(r)

Use the multi select box below to select one or more steps you want to examine


In [6]:
cq.exporters.export(r, "/tmp/x.stl")